In [ ]:
import os.path
css = ""
if os.path.isfile("style.html"):
    from IPython.core.display import HTML
    with open("style.html", "r") as file:
        css = file.read()
HTML(css)

# Hilfsfunktionen für die grafische Oberfläche

In diesem Notebook werden Hilfsfunktionen und Konstanten definiert, die für das Zeichnen und Spielen auf der grafischen Oberfläche benötigt werden.

In [ ]:
import math

Da das Spiel in der GUI in einem seperatem Thread läuft, werden Fehler oder Warnungen nicht in der Jupyter-Notebook geloggt. Deswegen wird zu Debugzwecken ein Datei-Logger implementiert. Dieser wird im Rahmen der Arbeit aber nicht weiter erläutert.

In [ ]:
import logging

logger = logging.getLogger('GUI')
logger.setLevel(logging.DEBUG)
fh = logging.FileHandler('log.txt')
fh.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
logger.addHandler(fh)

## Konstanten

Um eine einheitliche GUI zur Verfügung zu stellen, die auch leicht zu warten ist, werden im Folgenden einige Konstanten definiert.

### Spieler

Die Konstanten dienen dazu, die Strings der Spieler bzw. des leeren Feldes zu definieren.

In [ ]:
NO_PLAYER = ' '
PLAYER_1 = 'w'
PLAYER_2 = 'b'

### Brett

In der Abbildung sind alle Konstanten für die Zeichenfläche angegeben, um die Bedeutung der einzelnen Konstanten besser zeigen zu können. Zusätzlich sind in der Abbildung alle Farbkonstanten rotmarkiert.

![](../images/nmm-constants.svg)

In [ ]:
BOARD_SIZE = 500
CANVAS_PADDING = 30
ROW_WIDTH = 60
PLAYER_PIECE_RADIUS = 12
DEFAULT_PIECE_RADIUS = 5

DEFAULT_STONE_LINE_WIDTH = 1
SELECTED_STONE_LINE_WIDTH = 3
POUNDED_STONE_LINE_WIDTH = 5

STASH_STONES_SPACING = 5

STASH_HEIGHT = ((PLAYER_PIECE_RADIUS * 2 ) * 2) + (STASH_STONES_SPACING) + (CANVAS_PADDING * 2)
STASH_WIDTH = ((PLAYER_PIECE_RADIUS * 2 ) * 9) + (STASH_STONES_SPACING * 8) + (CANVAS_PADDING * 2)

CANVAS_HEIGHT = BOARD_SIZE
CANVAS_WIDTH = 1500

STASH_STARTING_POINT_X = BOARD_SIZE
STASH_STARTING_POINT_Y = CANVAS_HEIGHT - STASH_HEIGHT


### Farben

Die Farben für das Spiel werden im Folgenden definiert. In der Obigen Abbildung sind diese auch genauer erklärt.

In [ ]:
CANVAS_BACKGROUND_COLOR = '#ffffff'
BOARD_FOREGROUND_COLOR = '#191919'
BOARD_BACKGROUND_COLOR = '#ffffcb'
STASH_BACKGROUND_COLOR = '#dedede'
PLAYER_1_FILL_COLOR = '#E8E8E8'
PLAYER_1_STROKE_COLOR = '#191919'
PLAYER_2_FILL_COLOR = '#5c5c5c'
PLAYER_2_STROKE_COLOR = '#191919'
LAST_MOVED_COLOR = '#61aced'

### Text

In der GUI gibt es drei verschiedene Textarten:
- Spielnachricht (wer spielt gerade oder ob das Spiel beendet ist),
- Hinweise (falls zum Beispiel eine ungültige Aktion ausgeführt worden ist) und
- Informationen über den letzten Zug der künstlichen Intelligenz.

Diese Textarten haben eine eigene Schriftart, -größe und -farbe.

In [ ]:
TEXT_X = BOARD_SIZE + CANVAS_PADDING
TEXT_Y = CANVAS_PADDING
TEXT_MAX_WIDTH = CANVAS_WIDTH - BOARD_SIZE - 2 * CANVAS_PADDING
TEXT_VERTICAL_PADDING = 30
TEXT_MSG_FONT = '18px sans-serif'
TEXT_MSG_COLOR = '#333333'
TEXT_HINT_FONT = '14px sans-serif'
TEXT_HINT_COLOR = '#c75528'
TEXT_INFO_FONT = '14px mono'
TEXT_INFO_COLOR = '#333333'

### Schatten

Um das Spiel dynamischer zu gestalten, haben Spielsteine einen Schatten.

In [ ]:
SHADOW_COLOR_ENABLED = '#000000'
SHADOW_OFFSET_X_ENABLED = 2
SHADOW_OFFSET_Y_ENABLED = 2
SHADOW_BLUR_ENABLED = 2

SHADOW_COLOR_DISABLED = 'rgba(0, 0, 0, 0)'
SHADOW_OFFSET_X_DISABLED = 0
SHADOW_OFFSET_Y_DISABLED = 0
SHADOW_BLUR_DISABLED = 0

### Berechnete Werte

Die Koordinaten für die Knoten lassen sich aus den obrigen Konstanten berechnen. Da das Mühlespielbrett horizontal und vertikal identisch ist, werden für die Koordinaten auf der x- und y-Achse die gleichen Werte benötigt, die mit $av$ (für _available values_) bezeichnet werden. Es werden insgesamt sieben Werte $av_0$ bis $av_6$ benötigt, die in der folgenden Abbildung dargestellt werden.

![](../images/nmm-av.png)

Die Werte lassen sich wie folgt berechnen.

$$ av_0 =  CANVAS\_PADDING $$
$$ av_1 =  CANVAS\_PADDING + ROW\_WIDTH $$
$$ av_2 =  CANVAS\_PADDING + 2 \cdot ROW\_WIDTH $$
$$ av_3 =  \frac{BOARD\_SIZE}{2}$$
$$ av_4 =  BOARD\_SIZE - (CANVAS\_PADDING + 2 \cdot ROW\_WIDTH) $$
$$ av_5 =  BOARD\_SIZE - (CANVAS\_PADDING + ROW\_WIDTH)$$
$$ av_6 =  BOARD\_SIZE - CANVAS\_PADDING $$

In [ ]:
av = (
    math.floor(CANVAS_PADDING),
    math.floor(CANVAS_PADDING + ROW_WIDTH),
    math.floor(CANVAS_PADDING + 2 * ROW_WIDTH),
    math.floor(BOARD_SIZE / 2),
    math.floor(BOARD_SIZE - (CANVAS_PADDING + 2 * ROW_WIDTH)),
    math.floor(BOARD_SIZE - (CANVAS_PADDING + ROW_WIDTH)),
    math.floor(BOARD_SIZE - CANVAS_PADDING)
)


### Koordinaten

Die Koordinaten der Knoten sind in dem zweidimensionalen Tupel `coords` definiert. Zuerst wird der Ring definiert, von außen nach innen. Danach die Position im Ring, beginnend von oben links und dann im Uhrzeigersinn. In der Abbildung sind die Knoten mit den Koordinaten dargestellt. Die Werte von den Koordinaten sind die x- und y-Werte auf der Zeichenfläche, definiert in `av`.

![](images/nmm-coords.png)

In [ ]:
coords = (
    (
        (av[0], av[0]),
        (av[3], av[0]),
        (av[6], av[0]),
        (av[6], av[3]),
        (av[6], av[6]),
        (av[3], av[6]),
        (av[0], av[6]),
        (av[0], av[3])
    ),
    (
        (av[1], av[1]),
        (av[3], av[1]),
        (av[5], av[1]),
        (av[5], av[3]),
        (av[5], av[5]),
        (av[3], av[5]),
        (av[1], av[5]),
        (av[1], av[3])
    ),
    (
        (av[2], av[2]),
        (av[3], av[2]),
        (av[4], av[2]),
        (av[4], av[3]),
        (av[4], av[4]),
        (av[3], av[4]),
        (av[2], av[4]),
        (av[2], av[3])
    )
) 

## Funktionen zum Zeichnen

Die grafische Oberfläche (englisch _graphical user interface_, GUI) wird mit dem Python-Modul ipycanvas aufgebaut. Dieses Modul ermöglicht die Verwendung einer interaktiven Zeichenfläche zum Zeichnen von 2D-Objekten in IPython. Es bringt eine Reihe von Funktionen mit, um einfache Formen zeichnen zu können. Gezeichnet wird auf einem 2D-Canvas-Objekt mit den Startkoordinaten `(0,0)` oben links.

In [ ]:
import ipycanvas

Die Funktion `toggleShadow` schaltet den Schatten auf einem gegebenen Canvas ein und aus. Sie hat folgende Eingabeparameter:

- `c` ist eine Referenz auf ein Canvas-Objekt.
- `enable` ist ein boolischer Wert, der angibt, ob Schatten auf dem Canvas `c` ein oder ausgeschaltet werden soll.

Wird der Schatten eingeschaltet, werden die Schatteneigenschaften des Canvas mit den oben definierten Konstanten gesetzt. Andernfalls werden die Standardwerte von _ipycanvas_ gesetzt, was bedeutet, der Schatten wird deaktiviert. 

In [ ]:
def toggleShadow(c, enable):
    c.shadow_color    = SHADOW_COLOR_ENABLED    if enable else SHADOW_COLOR_DISABLED
    c.shadow_offset_x = SHADOW_OFFSET_X_ENABLED if enable else SHADOW_OFFSET_X_DISABLED
    c.shadow_offset_y = SHADOW_OFFSET_Y_ENABLED if enable else SHADOW_OFFSET_Y_DISABLED
    c.shadow_blur     = SHADOW_BLUR_ENABLED     if enable else SHADOW_BLUR_DISABLED

Die Funktion `drawCircle` dient zum Zeichnen eines Kreises auf einem Zeichenfeld. Die Funktion hat vier Argumente und drei optionale Parameter:

- `c` ist eine Referenz auf ein Canvas-Objekt, auf dem der Kreis gezeichnet werden soll.
- `coords` ist die Koordinate des Mittelpunktes des Kreies.
- `radius` ist der Radius des Kreises.
- `color` gibt die Farbe des Kreises an.
- `strokeColor` ist ein optionaler Parameter, der die Farbe der Umrandung angibt. Der Standardwert ist `None`. In dem Fall wird der Kreis nicht umrandet.
- `lineWidth` ist ein optionaler Parameter, der die Liniendicke angibt. Der Standardwert ist in der Kontante `DEFAULT_STONE_LINE_WIDTH` definiert.
- `useShadow` ist ein optionaler, boolischer Wert. Wenn er gesetzt ist, wird ein Schatten von dem Kreis gemalt. Standardmäßig ist der Wert `False`.

In [ ]:
def drawCircle(
        c,
        coords,
        radius,
        color,
        strokeColor = None,
        lineWidth = DEFAULT_STONE_LINE_WIDTH,
        useShadow = False):
    if useShadow:
        toggleShadow(c, True)
    c.fill_style = color
    c.fill_arc(coords[0], coords[1], radius, 0, 2 * math.pi)
    if useShadow:
        toggleShadow(c, False)
    if strokeColor is not None:
        c.line_width = lineWidth
        c.stroke_style = strokeColor
        c.stroke_arc(coords[0], coords[1], radius, 0, 2 * math.pi)

Die Funktion `drawStone` dient zum Zeichnen eines Steines auf einem Zeichenfeld mit Hilfe der Funktion `drawCircle`. Ein Spielstein besteht aus zwei Kreisen und ein leerer Knoten (also wo sich kein Spieler befinden) aus einem Kreis.

Die Funktion hat drei Argumente und zwei optionale Parameter:

- `c` ist eine Referenz auf ein Canvas-Objekt, auf dem der Stein gezeichnet werden soll;
- `coords` ist die Koordinate des Mittelpunktes des Steines;
- `player` gibt  den Spieler an;
- `selected` ist ein optionaler, boolischer Wert, der angibt, ob ein Spielerstein ausgewählt ist oder nicht. Der Standardwert ist `False`;
- `lastMoved` ist ein optionaler, boolischer Wert, der angibt ob der zu zeichnende Spielerstein zuletzt bewegt worden ist. Der Standardwert ist `False`.

In [ ]:
def drawStone(c, coords, player, selected = False, lastMoved = False):
    if player == NO_PLAYER:
        drawCircle(c, coords, DEFAULT_PIECE_RADIUS, BOARD_FOREGROUND_COLOR)
    else:
        color       = PLAYER_1_FILL_COLOR   if player == PLAYER_1 else PLAYER_2_FILL_COLOR
        strokeColor = PLAYER_1_STROKE_COLOR if player == PLAYER_1 else PLAYER_2_STROKE_COLOR
        
        lineWidth = SELECTED_STONE_LINE_WIDTH if selected else DEFAULT_STONE_LINE_WIDTH
        drawCircle(
            c,
            coords,
            PLAYER_PIECE_RADIUS,
            LAST_MOVED_COLOR if lastMoved else color,
            strokeColor,
            lineWidth,
            useShadow = True)
        drawCircle(
            c,
            coords,
            math.floor(PLAYER_PIECE_RADIUS / 2),
            color,
            strokeColor,
            lineWidth)

Die Funktion `drawPoundedStone` dient zum Zeichnen eines geschlagenden Steines auf einem Zeichenfeld. Ein geschlagender Stein wird durch ein Kreuz in der GUI dargestellt.

Die Funktion hat drei Argumente:

- `c` ist eine Referenz auf ein Canvas-Objekt, auf dem der geschlagende Stein gezeichnet werden soll;
- `coords` ist die Koordinate des Mittelpunktes des geschlagenden Steines;
- `player` gibt  den Spieler des Steines an.

In [ ]:
def drawPoundedStone(c, coords, player):
    x, y = coords
    offset = int(math.sin(0.25*math.pi) * PLAYER_PIECE_RADIUS)
    c.line_width = POUNDED_STONE_LINE_WIDTH
    c.stroke_style = PLAYER_1_FILL_COLOR   if player == PLAYER_1 else PLAYER_2_FILL_COLOR
    with ipycanvas.hold_canvas(c):
        c.stroke_line(x - offset, y - offset, x + offset, y + offset)
        c.stroke_line(x - offset, y + offset, x + offset, y - offset)
    

Die Funktion `drawText` zeichnet einen Text auf einer Zeichenfläche. Die Funktion hat zwei Argumente und zwei optionale Parameter:

- `c` ist eine Referenz auf ein Canvas-Objekt, auf dem der Text gezeichnet werden soll.
- `msg` ist die Nachricht, die auf dem Canvas geschrieben werden soll.
- `hint` ist ein optionaler String, der ein Hinweis oder eine Warnung ist. Standardmäßig ist die Variable auf `None` gesetzt.
- `information` ist ein optionales Dictionary. Alle Einträge des Dictionaries werden als Information auf dem Zeichenfeld ausgegeben.

Bei jedem Funktionsaufruf wird am Anfang der Inhalt der Zeichenfläche gelöscht, sodass sich immer nur eine Version der Texte auf der Zeichenfläche befindet.  

In [ ]:
def drawText(c, msg, hint = None, information = None):
    with ipycanvas.hold_canvas(c):
        c.clear()
        y = TEXT_Y
        c.font = TEXT_MSG_FONT
        c.fill_style = TEXT_MSG_COLOR
        c.fill_text(msg, TEXT_X, y, max_width = TEXT_MAX_WIDTH)
        y += TEXT_VERTICAL_PADDING
        if hint is not None:
            c.font = TEXT_HINT_FONT
            c.fill_style = TEXT_HINT_COLOR
            c.fill_text('Hint: ' + hint, TEXT_X, y, max_width = TEXT_MAX_WIDTH)
            y += TEXT_VERTICAL_PADDING
        if information is not None:
            c.font = TEXT_INFO_FONT
            c.fill_style = TEXT_INFO_COLOR
            c.fill_text('Information from last move:', TEXT_X, y, max_width = TEXT_MAX_WIDTH)
            y += TEXT_VERTICAL_PADDING
            for (key, value) in information.items():
                c.fill_text(f'   {key.ljust(16)} {value}', TEXT_X, y, max_width = TEXT_MAX_WIDTH)
                y += TEXT_VERTICAL_PADDING

Die Funktion `constructLine` dient zum Konstruieren einer Linie auf einem Zeichenfeld. Die Funktion hat drei Eingabeparameter:

- `c` ist eine Referenz auf ein Canvas-Objekt, auf dem die Linie gezeichnet werden soll.
- `start` ist die Koordinate des Startpunktes der Linie.
- `end` ist die Koordinate des Endpunktes der Linie.

Die Funktion aktualisiert einen Pfad auf dem Zeichenfeld, aber sie zeichnet noch nicht den aktualisierten Pfad.

In [ ]:
def constructLine(c, start, end):
    c.move_to(start[0], start[1])
    c.line_to(end[0], end[1])

Die Funktion `constructSquare` konstruiert ein Quadrat auf einer Zeichenfläche und lässt es mit Hilfe der Funktion `constructLine` zeichnen. Die Funktion hat zwei Eingabeargumente:

- `c` ist eine Referenz auf ein Canvas-Objekt, auf dem das Quadrat gezeichnet werden soll.
- `ring` ist ein Acht-Tupel, das die Koordinaten eines Ringes enthält.

In [ ]:
def constructSquare(c, ring):
    for i in range(4):
        start = i * 2
        end = (i * 2 + 2) if (i * 2 + 2 <= 6) else 0 
        constructLine(c, ring[start], ring[end])

Die Funktion `constructCrossLines` konstruiert die Querlinien des Mühlespiels auf einer Zeichenfläche und lässt es mit Hilfe der Funktion `constructLine` zeichnen. Die Funktion hat zwei Eingabeargumente:

- `c` ist eine Referenz auf ein Canvas-Objekt, auf dem die Querlinien gezeichnet werden sollen.
- `coords` ist ein zweidimensionales Tupel, welches alle Koordinaten des Spielbrettes enthält (vgl. das Kapitel _Koordinaten_ in der GUI).

In [ ]:
def constructCrossLines(c, coords):
    for i in range(4):
        k = i * 2 + 1
        constructLine(c, coords[0][k], coords[2][k])

Die Funktion `setupCanvas` erstellt das Canvas-Objekt und zeichnet den Hintergrund des Spielfeldes. Die Funktion hat keine Eingabeparameter und gibt eine Referenz auf das erstellte Canvas-Objekt zurück.

Die Zeichenfläche besteht aus einem MultiCanvas-Objekt mit drei Ebenen:

- Der Hintergrund, der das Spielbrett mit den Linien darstellt;
- Auf der zweiten Ebene wird der Text für das Spiel geschrieben;
- Die Spielsteine werden auf der obersten Ebene gezeichnet. 

In [ ]:
def setupCanvas():
    canvas = ipycanvas.MultiCanvas(3, width = CANVAS_WIDTH, height = CANVAS_HEIGHT)
    with ipycanvas.hold_canvas(canvas[0]):
        
        canvas[0].fill_style = CANVAS_BACKGROUND_COLOR
        canvas[0].fill_rect(0, 0, CANVAS_WIDTH, CANVAS_HEIGHT)
        
        canvas[0].fill_style = BOARD_BACKGROUND_COLOR
        canvas[0].fill_rect(0, 0, BOARD_SIZE, BOARD_SIZE)

        canvas[0].fill_style = STASH_BACKGROUND_COLOR
        canvas[0].fill_rect(STASH_STARTING_POINT_X, STASH_STARTING_POINT_Y, STASH_WIDTH, STASH_HEIGHT)


        canvas[0].stroke_style = BOARD_FOREGROUND_COLOR
        canvas[0].begin_path()

        for i in range(3):
            constructSquare(canvas[0], coords[i])

        constructCrossLines(canvas[0], coords)
        canvas[0].stroke()
    
    return canvas

Die Funktion `updateGui` dient zum Aktualisieren der Zeichenfläche für einen gegebenen Spielzustand. Die Funktion hat zwei Argumente und drei optionale Parameter:

- `c` ist eine Referenz auf ein Canvas-Objekt;
- `state` ist der Spielzustand, der in der GUI angezeigt werden soll;
- `selectedStone` ist eine Koordinate von einem selektierten Stein. Der Standardwert ist `None`. Ist in der Phase 2 oder 3 ein Stein auswählt, kann er mit diesem optionalen Parameter auf der Zeichenfläche hervorgehoben werden;
- `movedStone` ist die Koordinate des zuletzt bewegten Steins, um diesen in der GUI entsprechend zu markieren. Der Standardwert ist `None`, was bedeutet, das kein Stein bewegt worden ist;
- `poundedStones` ist eine Menge von den geschlagenden Steinen des Gegenspielers. Diese werden in der GUI als ein Kreuz dargestellt. Standardmäßig ist `poundedStones` eine leere Menge, was bedeutet, das kein Stein geschlagen worden ist. 

In [ ]:
def updateGui(c, state, selectedStone = None, movedStone = None, poundedStones = set()):
    with ipycanvas.hold_canvas(c):
        c.clear()
        ((stashP1, stashP2), squares) = state

        # update pieces on the board
        for i in range(len(squares)):
            for j in range(len(squares[i])):
                drawStone(
                    c,
                    coords[i][j],
                    squares[i][j],
                    selected = selectedStone == (i, j),
                    lastMoved = movedStone == (i, j))

        for (player, (i, j)) in poundedStones:
            drawPoundedStone(c, coords[i][j], player)
            
        # update pieces on the stash

        # player 1

        x = STASH_STARTING_POINT_X + PLAYER_PIECE_RADIUS
        y = STASH_STARTING_POINT_Y + CANVAS_PADDING + PLAYER_PIECE_RADIUS

        for i in range(stashP1):
            x += 2 * PLAYER_PIECE_RADIUS + STASH_STONES_SPACING
            drawStone(c, (x, y), PLAYER_1)

        # player 2

        x = STASH_STARTING_POINT_X + PLAYER_PIECE_RADIUS
        y = STASH_STARTING_POINT_Y + CANVAS_PADDING + 3 * PLAYER_PIECE_RADIUS + STASH_STONES_SPACING

        for i in range(stashP2):
            x += 2 * PLAYER_PIECE_RADIUS + STASH_STONES_SPACING
            drawStone(c, (x, y), PLAYER_2)
    

## Hilfsfunktionen für das Spielen in der GUI

In diesem Kapitel werden Funktionen deklariert, die Hilfsfunktionen für die GUI darstellen, aber unabhängig von dem eigentlichen Spielzustand sind.

Zusätzlich werden die Jupyter-Notebooks von dem Minimax- und Alpha-Beta-Pruning-Algorithmus benötigt und hier ausführt.

In [ ]:
%run ./nmm-minimax.ipynb
%run ./nmm-alpha-beta-pruning.ipynb

Die Funktion `getClickedStone` dient zum Ermitteln, ob auf der Zeichenfläche eine Ecke angeklickt worden ist, auf dem ein Stein stehen kann. Die Funktion hat zwei Argumente:

- `x` für den Wert auf der horizontalen Achse;
- `y` für den Wert auf der vertikalen Achse.

Es müssen nicht die genauen Werte für x und y angeklickt werden, sondern es gibt einen Puffer in Höhe des Radius von einem Spielerstein. Falls eine Position für einen Stein angeklickt worden ist, für die jeweilige Koordinate aus dem `coords`-Tupel zurückgegeben. Falls keine Position gefunden worden ist, wird `None` zurückgegeben.

In [ ]:
def getClickedStone(x, y):
    for value in av:
        if value - PLAYER_PIECE_RADIUS <= x <= value + PLAYER_PIECE_RADIUS:
            x = value
        if value - PLAYER_PIECE_RADIUS <= y <= value + PLAYER_PIECE_RADIUS:
            y = value

    for i in range(len(coords)):
        for j in range(len(coords[i])):
            if coords[i][j] == (x, y):
                return (i, j)
    return None

Die Funktion `getChangedStones` ermittelt den zuletzt bewegten Stein eines Spielers und alle geschlagenden Steine des Gegenspielers zwischen zwei Zuständen. Die Funktion hat drei Argumente:

- `oldState` ist der Ausgangszustand;
- `newState` ist der neue Zustand;
- `player` ist der Spieler, den den Zug gespielt hat.

Die Funktion gibt ein Zwei-Tupel der Form `<movedStone, poundedStones>` mit
- `movedStone` ist die Koordinate des bewegten Steins des Spielers;
- `poundedStones` ist eine Menge von Zwei-Tupeln der Form `<op, coord>` mit 
    - `op` ist der Gegenspieler;
    - `coord` ist die Koordinate des geschlagenden Steines;

zurück.

In [ ]:
def getChangedStones(oldState, newState, player):
    (_, oldBoard) = oldState
    (_, newBoard) = newState
    op = opponent(player)
    movedStone = None;
    poundedStones = set()
    
    for i in range(len(oldBoard)):
        for j in range(len(oldBoard[i])):
            if oldBoard[i][j] != newBoard[i][j]:
                if newBoard[i][j] == player:
                    movedStone = (i, j)
                if oldBoard[i][j] == op:
                    poundedStones |= { (op, (i, j)) }
    return (movedStone, poundedStones)